In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import pandas
import numpy as np



In [ ]:
# from google.colab import files
# src = list(files.upload().values())[0]
# open('trepan.py','wb').write(src)
import trepan
from trepan import Trepan,Oracle


In [ ]:
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
def load_landsat_data(filename):
	'''
	Utility function to load Landsat dataset.
	Landsat dataset : https://archive.ics.uci.edu/ml/datasets/Statlog+(Landsat+Satellite)
	num_classes= 7, but 6th is empty. 
	This functions
	- Reads the data
	- Renames the class 7 to 6
	- Generates one-hot vector labels
	'''

	data = pd.read_csv('data/diabetes.csv', delimiter=',')
	features = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]
	data[["Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]] = data[["Glucose","BloodPressure","SkinThickness","Insulin","BMI","DiabetesPedigreeFunction","Age"]].replace(0,data.mean(axis=0))
	outcome = data.iloc[:,8]
	
	d = preprocessing.normalize(data.iloc[:,0:8], axis=0)
	scaled_df = pd.DataFrame(d, columns = features)
	scaled_df = pd.concat([scaled_df, outcome], axis=1)
	X = scaled_df.iloc[:,0:8].to_numpy()
	y = data.iloc[:,8].to_numpy()
	np.random.seed(42)
	x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=5)
	x_train_df = pd.DataFrame(x_train,columns = features)
	x_test_df = pd.DataFrame(x_test,columns = features)
	y_train_df = pd.DataFrame(y_train);
	y_test_df = pd.DataFrame(y_test)
 
	dataX = np.array(x_train_df.values)
	dataY = np.array(y_train_df.values);
	testX = np.array(x_test_df.values)
	testY = np.array(y_test_df.values);

	num_classes = 2
	# dataY_onehot = dataY
	dataY_onehot = np.zeros([dataY.shape[0], num_classes])
	for i in range(dataY.shape[0]):
		dataY_onehot[i][dataY[i]] = 1


	# dataY_onehot[np.arange(dataY_onehot.shape[0]), dataY] = 1

	return dataX, dataY_onehot, testX, testY

In [ ]:
# dataY = np.zeros((4, 1),dtype=int)
# dataY[0][0] = int(1)
# dataY[2][0] = int(1)
# dataY[:,0].astype(bool)
# num_classes = 2
# dataY_onehot = np.zeros([dataY.shape[0], num_classes])
# for i in range(dataY.shape[0]):
#   dataY_onehot[i][dataY[i]] = 1
# # dataY_onehot[np.arange(dataY_onehot.shape[0]), dataY] = 1
# print(dataY_onehot)

In [ ]:
def create_model (trainX,trainY,num_classes):

	model = Sequential();
	model.add(Dense(12,kernel_initializer = tf.keras.initializers.glorot_uniform(seed=6), input_shape=(8,), activation='relu'));
	model.add(Dense(8,kernel_initializer = tf.keras.initializers.glorot_uniform(seed=6), activation='relu'));
	# model.add(Dense(1,kernel_initializer = tf.keras.initializers.glorot_uniform(seed=6), activation='sigmoid'))
	# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	model.add(Dense(num_classes, activation="softmax"))
	model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

	model.fit(trainX, trainY, epochs=150, batch_size=10, verbose=0)
  
	return model

In [ ]:
#load training data
trainX, trainY, testX, testY = load_landsat_data("data/diabetes.csv")
# testX, testY = load_landsat_data("data/sat.tst")
num_classes = trainY.shape[1]
total_num_examples = trainX.shape[0]
print(num_classes,total_num_examples)

2 537


In [ ]:
print(trainX)

[[0.03529979 0.01265862 0.0305373  ... 0.02719637 0.03674434 0.03684448]
 [0.07059957 0.03711276 0.03743282 ... 0.03905399 0.01752711 0.03991486]
 [0.02823983 0.04257898 0.02955223 ... 0.03361471 0.00938952 0.02968028]
 ...
 [0.02823983 0.02790649 0.02955223 ... 0.03067751 0.02773039 0.02251607]
 [0.04235974 0.03596198 0.03841789 ... 0.03002479 0.03536721 0.05014943]
 [0.05647966 0.05638838 0.03743282 ... 0.04079456 0.03787108 0.0583371 ]]


In [ ]:
model = create_model(trainX,trainY,num_classes)
oracle = Oracle(model,num_classes,trainX)

#build tree with TREPAN
MIN_EXAMPLES_PER_NODE = 30
MAX_NODES=200

In [ ]:
root=Trepan.build_tree(MIN_EXAMPLES_PER_NODE,MAX_NODES,trainX,oracle)

1/1 [==============================] - 0s 21ms/step
NEW NODE! with priority = -0.3128491620111732
############PROCESSING 537 #############
ALL OK
SPLITTING 537 EXAMPLES
NEW NODE! with priority = -0.11173184357541903
NEW NODE! with priority = -0.033519553072625684
############PROCESSING 411 #############
ALL OK
SPLITTING 411 EXAMPLES
NEW NODE! with priority = -0.05959031657355676
NEW NODE! with priority = -0.027932960893854743
############PROCESSING 368 #############
ALL OK
SPLITTING 368 EXAMPLES
NEW NODE! with priority = -0.042830540037243924
NEW NODE! with priority = -0.01303538175046555
############PROCESSING 352 #############
ALL OK
SPLITTING 352 EXAMPLES
NEW NODE! with priority = -0.029795158286778433
NEW NODE! with priority = -0.01303538175046555
############PROCESSING 126 #############
ALL OK
SPLITTING 126 EXAMPLES
NEW NODE! with priority = -0.02048417132216015
NEW NODE! with priority = -0.013035381750465543
############PROCESSING 336 #############
ALL OK
SPLITTING 336 EXAMPLES
N

In [ ]:
print(root.leaf)

False


In [ ]:
print(root.splitrule.splits)

[(1, 'lte', 0.040565108005694274)]


In [ ]:
print(root.left_child.left_child.splitrule.splits)

[(6, 'lte', 0.06860611885801414)]


In [ ]:
def binaryTreePaths(root):
  res = []
  def dfs(node, s):
    
    if(type(node) != None):
      if s != "":
        s += "->"

      if node.leaf:
        s += str(node.dominant)
        res.append(s)
      if node.left_child: 
        split_cond_left = str("feature "+str(node.splitrule.splits[0][0]) + " "+ str(node.splitrule.splits[0][1]) +" "+ str(node.splitrule.splits[0][2]))
        s_left = s + split_cond_left
        dfs(node.left_child, s_left)
      if node.left_child: 
        split_cond_right = str("feature "+str(node.splitrule.splits[0][0]) + " not satisfy "+ str(node.splitrule.splits[0][1]) +" "+ str(node.splitrule.splits[0][2]))
        s_right = s + split_cond_right
        dfs(node.left_child, s_right)
  dfs(root, "") 
  return res
        

In [ ]:
binaryTreePaths(root)

['feature 1 lte 0.040565108005694274->feature 0 lte 0.056479656977492965->feature 6 lte 0.06860611885801414->feature 2 lte 0.04432833988171223->feature 4 lte 0.08017458454813198->feature 7 lte 0.0675482164024071->feature 6 lte 0.062409033304233655->0.0',
 'feature 1 lte 0.040565108005694274->feature 0 lte 0.056479656977492965->feature 6 lte 0.06860611885801414->feature 2 lte 0.04432833988171223->feature 4 lte 0.08017458454813198->feature 7 lte 0.0675482164024071->feature 6 not satisfy lte 0.062409033304233655->0.0',
 'feature 1 lte 0.040565108005694274->feature 0 lte 0.056479656977492965->feature 6 lte 0.06860611885801414->feature 2 lte 0.04432833988171223->feature 4 lte 0.08017458454813198->feature 7 not satisfy lte 0.0675482164024071->feature 6 lte 0.062409033304233655->0.0',
 'feature 1 lte 0.040565108005694274->feature 0 lte 0.056479656977492965->feature 6 lte 0.06860611885801414->feature 2 lte 0.04432833988171223->feature 4 lte 0.08017458454813198->feature 7 not satisfy lte 0.0675

In [ ]:
#calculate fidelity
num_test_examples= testX.shape[0]
correct=0
ann_arr = []
tree_arr = []
for i in range(0,num_test_examples):
	ann_prediction = oracle.get_oracle_label(testX[i,:])
	tree_prediction = root.classify(testX[i,:])
	# correct += (testY[i]==tree_prediction)
	ann_arr.append(ann_prediction)
	tree_arr.append(tree_prediction)
	correct += (ann_prediction==tree_prediction)

fidelity=float(correct)/num_test_examples

print("Fidelity of the model is : "+str(fidelity))

1/1 [==============================] - 0s 21ms/step
Fidelity of the model is : 0.9090909090909091


In [ ]:
print(ann_arr)

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [ ]:
print(tree_arr)

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
y_train_df = pd.DataFrame(trainY)

In [ ]:
x_train_df = pd.DataFrame(trainX)

In [ ]:
x_test_df = pd.DataFrame(testX)

In [ ]:
print(x_test_df)

           0         1         2         3         4         5         6  \
0    0.04236  0.026468  0.030537  0.040811  0.030155  0.034811  0.005321   
1    0.03530  0.037976  0.039403  0.026191  0.019098  0.029155  0.011643   
2    0.02118  0.030496  0.035463  0.026191  0.019098  0.028067  0.012958   
3    0.02824  0.028482  0.033493  0.048463  0.019098  0.035682  0.009077   
4    0.02824  0.027619  0.027582  0.021681  0.011727  0.022627  0.021283   
..       ...       ...       ...       ...       ...       ...       ...   
226  0.02118  0.037976  0.039403  0.026191  0.019098  0.037422  0.025164   
227  0.04236  0.030783  0.043343  0.026191  0.019098  0.040033  0.045508   
228  0.00706  0.029633  0.039403  0.014029  0.019625  0.021104  0.030735   
229  0.02824  0.024454  0.028567  0.028058  0.011727  0.030242  0.019155   
230  0.00706  0.044017  0.040388  0.053564  0.116074  0.044167  0.043004   

            7  
0    0.047079  
1    0.070619  
2    0.027633  
3    0.033774  
4    0.

In [ ]:
print(x_test_df.describe().T)

   count      mean       std       min       25%       50%       75%       max
0  231.0  0.028973  0.023487  0.000000  0.007060  0.021180  0.042360  0.120019
1  231.0  0.035287  0.008970  0.016399  0.028482  0.033660  0.041284  0.056964
2  231.0  0.036190  0.005634  0.023642  0.032015  0.035463  0.039403  0.053194
3  231.0  0.033497  0.012529  0.010203  0.026191  0.026191  0.040811  0.080347
4  231.0  0.027435  0.024011  0.003351  0.019098  0.019098  0.025728  0.178059
5  231.0  0.034962  0.007681  0.020996  0.029209  0.035029  0.039815  0.064619
6  231.0  0.030407  0.022930  0.005321  0.015962  0.023662  0.038466  0.151484
7  231.0  0.035343  0.012531  0.021493  0.025586  0.030704  0.042985  0.070619


In [ ]:
print(x_train_df.describe().T)

   count      mean       std       min       25%       50%       75%       max
0  537.0  0.026360  0.023897  0.000000  0.007060  0.021180  0.042360  0.105899
1  537.0  0.034887  0.008668  0.012659  0.028770  0.033948  0.039990  0.057251
2  537.0  0.035329  0.006092  0.011821  0.031522  0.034478  0.038418  0.060090
3  537.0  0.034119  0.012183  0.008927  0.026191  0.029333  0.040811  0.126259
4  537.0  0.028813  0.021498  0.003590  0.019098  0.019098  0.032309  0.202471
5  537.0  0.035448  0.007394  0.019799  0.030134  0.034811  0.039815  0.072995
6  537.0  0.029164  0.019735  0.004883  0.014898  0.023161  0.039248  0.143222
7  537.0  0.033452  0.011783  0.021493  0.024563  0.029680  0.039915  0.082900


##Conditions

In [ ]:
index_arr = x_train_df[(x_train_df.iloc[:,1] > 0.039)&(x_train_df.iloc[:,5] > 0.037)].index

In [ ]:
print(index_arr.size)

69


In [ ]:
y_train_clean_data = y_train_df[y_train_df.index.isin(index_arr)]

In [ ]:
print(y_train_clean_data)

       0    1
3    1.0  0.0
6    1.0  0.0
23   0.0  1.0
29   0.0  1.0
48   0.0  1.0
..   ...  ...
477  0.0  1.0
479  0.0  1.0
488  1.0  0.0
497  0.0  1.0
536  0.0  1.0

[69 rows x 2 columns]


In [ ]:
y_train_clean_data.iloc[:,1].sum()

54.0

###high risk

standard we use:

x_test_df[(x_test_df.iloc[:,1] > 0.05)].index 0/20

x_test_df[(x_test_df.iloc[:,1] > 0.05) & (x_test_df.iloc[:,0] > 0.04)].index                       0/7

x_test_df[(x_test_df.iloc[:,1] > 0.05) & (x_test_df.iloc[:,5] > 0.039)].index                            0/9

x_test_df[(x_test_df.iloc[:,1] > 0.033) & (x_test_df.iloc[:,5] > 0.039)& (x_test_df.iloc[:,6] > 0.038)].index      3/17

 x_test_df[(x_test_df.iloc[:,1] > 0.033) & (x_test_df.iloc[:,5] > 0.039)& (x_test_df.iloc[:,3] > 0.026) & (x_test_df.iloc[:,7] > 0.030)].index                          9/28
 

###low risk

x_test_df[(x_test_df.iloc[:,2] > 0.045)].index.         5/12


x_test_df[(x_test_df.iloc[:,2] > 0.042)&(x_test_df.iloc[:,3] > 0.040)].index        4/9

x_test_df[(x_test_df.iloc[:,2] > 0.040)&(x_test_df.iloc[:,3] > 0.035)& (x_test_df.iloc[:,4] > 0.025)].index.      4/11


 x_test_df[(x_test_df.iloc[:,2] > 0.038)&(x_test_df.iloc[:,3] > 0.033)& (x_test_df.iloc[:,4] > 0.023) & (x_test_df.iloc[:,6] > 0.038)].index                  1/7

  x_test_df[(x_test_df.iloc[:,2] > 0.038)&(x_test_df.iloc[:,3] > 0.033)& (x_test_df.iloc[:,4] > 0.023) & (x_test_df.iloc[:,7] > 0.042)].index             3/5



In [ ]:
row_num_double = x_test_df[(x_test_df.iloc[:,2] > 0.038)&(x_test_df.iloc[:,3] > 0.033)& (x_test_df.iloc[:,4] > 0.023) & (x_test_df.iloc[:,7] > 0.042)].index
print(row_num_double.shape[0])

5


In [ ]:
ind = row_num_double.tolist()

In [ ]:
tree_result = []
ann_result = []
for i in range (0,row_num_double.shape[0]):
  ind = row_num_double[i]
  tree_result.append(tree_arr[ind])
  ann_result.append(ann_arr[ind])
  

In [ ]:
print(ann_result)

[1, 0, 0, 1, 1]


In [ ]:
print(tree_result)

[1.0, 1.0, 0.0, 1.0, 1.0]


In [ ]:
sum = 0
for i in range(testY.shape[0]):
  sum += testY[i][0]
                  
print(sum)

71


In [ ]:
print(testY.shape[0])

231


In [ ]:
(231-71)/231

0.6926406926406926